# Capstone Project: Data Collection     

29/11/2020

We collect restaurant data via the Foursquare API.

In [1]:
import pandas as pd
import numpy as np
import requests 
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!pip install folium
import folium # map rendering library


In [7]:
# Coordinates of Copenhagen
latitude = 55.6761
longitude = 12.5683

# create map of Toronto using latitude and longitude values
map_copenhagen = folium.Map(location=[latitude, longitude], zoom_start=10)
    
map_copenhagen

In [8]:
# @hidden_cell
CLIENT_ID = '34CRGBIL11CRLO5BNFSRKHWNRQT1GWTB4QKGYG5GLQYM3JKQ' # your Foursquare ID
CLIENT_SECRET = 'HRPY3HMJC1FSPYTC2VRVUANCY1JY20NBRA5400RQYOEOFCNB' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
VERSION = '20201123' # Foursquare API version
RADIUS=500
LIMIT = 50 # A default Foursquare API limit value

In [9]:
# List of categories
afghan = '503288ae91d4c4b30a586d67'
african = '4bf58dd8d48988d1c8941735'
american = '4bf58dd8d48988d14e941735'
asian = '4bf58dd8d48988d142941735'
australian = '4bf58dd8d48988d169941735'
austrian = '52e81612bcbc57f1066b7a01'
bangladesh = '5e179ee74ae8e90006e9a746'
belgium = '52e81612bcbc57f1066b7a02'
cajun = '4bf58dd8d48988d17a941735'
caribbean = '4bf58dd8d48988d144941735'
caucasian = '5293a7d53cf9994f4e043a45'
czech = '52f2ae52bcbc57f1066b8b81'
dutch = '5744ccdfe4b0c0459246b4d0'
eastEurope = '4bf58dd8d48988d109941735'
english = '52e81612bcbc57f1066b7a05'
french = '4bf58dd8d48988d10c941735'
german = '4bf58dd8d48988d10d941735'
greek = '4bf58dd8d48988d10e941735'
hungary = '52e81612bcbc57f1066b79fa'
indian = '4bf58dd8d48988d10f941735'
italian = '4bf58dd8d48988d110941735'
jewish = '52e81612bcbc57f1066b79fd'
latin = '4bf58dd8d48988d1be941735'
meditarranean = '4bf58dd8d48988d1c0941735'
mexican = '4bf58dd8d48988d1c1941735'
middleEast = '4bf58dd8d48988d115941735'
modernEuropean = '52e81612bcbc57f1066b79f9'
pakistan = '52e81612bcbc57f1066b79f8'
polish = '52e81612bcbc57f1066b7a04'
portugal = '4def73e84765ae376e57713a'
russia = '5293a7563cf9994f4e043a44'
scandinavia = '4bf58dd8d48988d1c6941735'
scotland = '5744ccdde4b0c0459246b4a3'
slovak = '56aa371be4b08b9a8d57355a'
spain = '4bf58dd8d48988d150941735'
srilanka = '5413605de4b0ae91d18581a9'
swiss = '4bf58dd8d48988d158941735'
turkey = '4f04af1f2fb6e1c99f3db0bb'
ukraine = '52e928d0bcbc57f1066b7e96'

category_list = [afghan,african,american,asian,australian,austrian,bangladesh,belgium,cajun,caribbean,caucasian,czech,dutch,eastEurope,english,
                 french,german,greek,hungary,indian,italian,jewish,latin,meditarranean,mexican,middleEast,modernEuropean,pakistan,polish,
                 portugal,russia,scandinavia,scotland,slovak,spain,srilanka,swiss,turkey,ukraine] 

CATEGORIES = ','.join(category_list)

In [10]:
def getVenues(latitude, longitude, RADIUS=500):
    
    url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse'.format(
            CATEGORIES,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            RADIUS,
            LIMIT)
    
    response = requests.get(url)
    results = response.json()
    
    if response.status_code == 200:
        venues = results['response']['venues']
    else:
        venues = []
        print("HTTP-{}".format(response.status_code))
        
    venue_list = []
    
    number_of_venues = len(venues)
    
    for i in range(number_of_venues):
        
        try:
            venue_id = venues[i]['id']
            venue_name = venues[i]['name']
            lat = venues[i]['location']['lat']
            lng = venues[i]['location']['lng']
            category = venues[i]['categories'][0]['name']
            
            record = (latitude, longitude, number_of_venues, i, venue_id, venue_name, lat, lng, category)
        except:
            record = (latitude, longitude, number_of_venues, i, '', '', '', '', '')
        
        venue_list.append(record)
        
        
    return( venue_list )

# Copenhagen

In [13]:
# Coordinates of Copenhagen
latitude = 55.6761
longitude = 12.5683

# Box boundaries
y_max = latitude + 0.05
y_min = latitude - 0.06
x_max = longitude + 0.1
x_min = longitude - 0.1


lat_centre = (y_max+y_min)/2.0

y_step = 1.0/(110.574)
x_step = 1.0/(111.32*np.cos(2.0*np.pi*lat_centre/360.0))

num_x_points = int((x_max-x_min)/x_step)
num_y_points = int((y_max-y_min)/y_step)

lats = np.linspace(x_min,x_max,num_x_points)
longs = np.linspace(y_min,y_max,num_y_points)

' Create matrix of coordinate values '
coord_list = []
for x in longs:
    for y in lats:
        
        coord_list.append( (x,y) )

print("Number of points = {}".format(len(coord_list)))

Number of points = 144


In [14]:
# Coordinates of Copenhagen
latitude = 55.6761
longitude = 12.5683

# create map of Toronto using latitude and longitude values
map_copenhagen = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_copenhagen)  
    
map_copenhagen

## Retrieve data from Foursquare

In [15]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('copenhagen_20201210_2330.csv',encoding='utf-8')

lat = 55.6161 long = 12.4683 Number of Venues = 0
lat = 55.6161 long = 12.4864818182 Number of Venues = 1
lat = 55.6161 long = 12.5046636364 Number of Venues = 0
lat = 55.6161 long = 12.5228454545 Number of Venues = 0
lat = 55.6161 long = 12.5410272727 Number of Venues = 0
lat = 55.6161 long = 12.5592090909 Number of Venues = 0
lat = 55.6161 long = 12.5773909091 Number of Venues = 0
lat = 55.6161 long = 12.5955727273 Number of Venues = 0
lat = 55.6161 long = 12.6137545455 Number of Venues = 0
lat = 55.6161 long = 12.6319363636 Number of Venues = 0
lat = 55.6161 long = 12.6501181818 Number of Venues = 0
lat = 55.6161 long = 12.6683 Number of Venues = 0
lat = 55.6261 long = 12.4683 Number of Venues = 0
lat = 55.6261 long = 12.4864818182 Number of Venues = 0
lat = 55.6261 long = 12.5046636364 Number of Venues = 1
lat = 55.6261 long = 12.5228454545 Number of Venues = 0
lat = 55.6261 long = 12.5410272727 Number of Venues = 0
lat = 55.6261 long = 12.5592090909 Number of Venues = 0
lat = 55.6

In [16]:
print("Limit breached in {} sites".format(sum(df_venues['num_venues']==50)/50))
np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0)

Limit breached in 5 sites


array([[55.6661    , 12.55920909],
       [55.6761    , 12.55920909],
       [55.6761    , 12.57739091],
       [55.6761    , 12.59557273],
       [55.6861    , 12.55920909]])

## Calculate new site locations at breached points

In [17]:
zoom_coord_list = np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0).tolist()

In [18]:
coord_list = []

for coord in zoom_coord_list:
    
    x = coord[1]
    y = coord[0]
    
    y_step = 1.0/(110.574)
    x_step = 1.0/(111.32*np.cos(2.0*np.pi*y/360.0))

    xleft = x - (x_step/4.0)
    xright = x + (x_step/4.0)
    ytop = y + (y_step/4.0)
    ybottom = y - (y_step/4.0)
    
    coord_list.append( (y,x) )
    coord_list.append( (ytop,xleft) )
    coord_list.append( (ybottom,xleft) )
    coord_list.append( (ytop,xright) )
    coord_list.append( (ybottom,xright) )

In [19]:
# Coordinates of Copenhagen
latitude = 55.6761
longitude = 12.5683

# create map of Toronto using latitude and longitude values
map_copenhagen = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_copenhagen)  
    
map_copenhagen

## Retrieve info from Foursquare

In [20]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng, RADIUS=250)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('copenhagen_20201210_2332.csv',encoding='utf-8')

lat = 55.6661 long = 12.5592090909 Number of Venues = 7
lat = 55.6683609293 long = 12.5552273191 Number of Venues = 37
lat = 55.6638390707 long = 12.5552273191 Number of Venues = 3
lat = 55.6683609293 long = 12.5631908627 Number of Venues = 4
lat = 55.6638390707 long = 12.5631908627 Number of Venues = 8
lat = 55.6761 long = 12.5592090909 Number of Venues = 11
lat = 55.6783609293 long = 12.5552263013 Number of Venues = 3
lat = 55.6738390707 long = 12.5552263013 Number of Venues = 50
lat = 55.6783609293 long = 12.5631918805 Number of Venues = 21
lat = 55.6738390707 long = 12.5631918805 Number of Venues = 47
lat = 55.6761 long = 12.5773909091 Number of Venues = 31
lat = 55.6783609293 long = 12.5734081195 Number of Venues = 50
lat = 55.6738390707 long = 12.5734081195 Number of Venues = 18
lat = 55.6783609293 long = 12.5813736987 Number of Venues = 31
lat = 55.6738390707 long = 12.5813736987 Number of Venues = 5
lat = 55.6761 long = 12.5955727273 Number of Venues = 12
lat = 55.6783609293 lo

In [21]:
print("Limit breached in {} sites".format(sum(df_venues['num_venues']==50)/50))
np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0)

Limit breached in 2 sites


array([[55.67383907, 12.5552263 ],
       [55.67836093, 12.57340812]])

## Calculate new site locations at breached points

In [22]:
zoom_coord_list = np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0).tolist()

In [23]:
coord_list = []

for coord in zoom_coord_list:
    
    x = coord[1]
    y = coord[0]
    
    y_step = 1.0/(110.574)
    x_step = 1.0/(111.32*np.cos(2.0*np.pi*y/360.0))

    xleft = x - (x_step/8.0)
    xright = x + (x_step/8.0)
    ytop = y + (y_step/8.0)
    ybottom = y - (y_step/8.0)
    
    coord_list.append( (y,x) )
    coord_list.append( (ytop,xleft) )
    coord_list.append( (ybottom,xleft) )
    coord_list.append( (ytop,xright) )
    coord_list.append( (ybottom,xright) )

In [24]:
# Coordinates of Copenhagen
latitude = 55.6761
longitude = 12.5683

# create map of Toronto using latitude and longitude values
map_copenhagen = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_copenhagen)  
    
map_copenhagen

## Retrieve info from Foursquare

In [25]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng, RADIUS=100)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('copenhagen_20201210_2335.csv',encoding='utf-8')


lat = 55.6738390707 long = 12.5552263013 Number of Venues = 5
lat = 55.6749695353 long = 12.5532350216 Number of Venues = 2
lat = 55.672708606 long = 12.5532350216 Number of Venues = 10
lat = 55.6749695353 long = 12.557217581 Number of Venues = 1
lat = 55.672708606 long = 12.557217581 Number of Venues = 21
lat = 55.6783609293 long = 12.5734081195 Number of Venues = 17
lat = 55.679491394 long = 12.5714166096 Number of Venues = 2
lat = 55.6772304647 long = 12.5714166096 Number of Venues = 18
lat = 55.679491394 long = 12.5753996294 Number of Venues = 28
lat = 55.6772304647 long = 12.5753996294 Number of Venues = 18


## Combine all files into single file

In [26]:
import os
files = os.listdir('.')

file_pattern = 'copenhagen_'

venue_files = []

' Obtain all files holding venue info '
for filename in files:
    if file_pattern in filename:
        venue_files.append(filename)

' Read in files into dataframe '
df_venues = pd.DataFrame()
for filename in venue_files:
    df = pd.read_csv(filename,encoding='utf-8')
    print( "File: {} Records: {}".format(filename,len(df)) )
    df_venues = df_venues.append(df, ignore_index = True) 

File: copenhagen_20201210_2330.csv Records: 1016
File: copenhagen_20201210_2332.csv Records: 560
File: copenhagen_20201210_2335.csv Records: 122


## Remove duplicate entries and clean up data

In [27]:
df_venues.sort_values("venue_id", inplace = True)
df_venues.drop_duplicates(subset ="venue_id", keep = False, inplace = True)
df_venues.to_csv('copenhagen_all.csv',encoding='utf-8')

In [28]:
' Keep only venues with "Restaurant" in their category title '
df_venues['restaurant'] = df_venues['category'].apply(lambda x: True if 'Restaurant' in x else False)
df_venues = df_venues[df_venues['restaurant'] == True]
df_venues.drop('restaurant', axis=1, inplace=True)
df_venues.to_csv('copenhagen_restaurants.csv',encoding='utf-8')

In [29]:
from folium.plugins import FastMarkerCluster

# Coordinates of Copenhagen
latitude = 55.6761
longitude = 12.5683

# create map of Zurich using latitude and longitude values
map_copenhagen = folium.Map(location=[latitude, longitude], zoom_start=10)

FastMarkerCluster(data=list(zip(df_venues['lat_venue'],df_venues['long_venue']))).add_to(map_copenhagen)

folium.LayerControl().add_to(map_copenhagen);

map_copenhagen
